In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import torch as T
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torchvision import transforms
 
device  = T.device("cpu")
T.manual_seed(42)

print('Good imports')

Good imports


In [2]:
# Read data
df = pd.read_csv('data.csv', index_col=False)

# Extract labels
labels_unordered = np.array(df[['784']])
# Need labels used to be numbered correctly to call to_categorical()
labels = np.copy(labels_unordered)
labels = np.where(labels == 49, 10, labels)
labels = np.where(labels == 50, 11, labels)
labels = np.where(labels == 51, 12, labels)
labels = np.where(labels == 52, 13, labels)
labels = np.where(labels == 82, 14, labels)
labels = np.where(labels == 33, 15, labels)
labels = np.where(labels == 34, 16, labels)


# Drop label col from data
df.drop(columns=['784'], inplace=True)

# Transomform data from 784x1 to 28x28 shape
data2D = []
tmp_data = df.values
for img in tqdm(tmp_data):
    data2D.append(img.reshape(1,28,28))
data2D = np.array(data2D)

100%|██████████| 240520/240520 [00:00<00:00, 705234.52it/s]


In [18]:
class TorchDataSet(Dataset):
    def __init__(self, data, labels, transforms=None):
        self.X = data
        self.labels = labels
        self.transforms = transforms
        
    def __getitem__(self, i):
        data = self.X.iloc[i,:]
        data = np.array(data).astype(np.uint8).reshape(1, 28, 28) 
        
        # apply transforms if there are any
        if self.transforms:
            data = self.transforms(data)
        
        # return tuple of data + label if given a label set
        if self.labels is not None:
            return (data, self.y[i])
        
        else:
            return data
    
    def __len__(self):
        return len(self.data)

In [19]:
# Split data & load into custum torch dataset
data_train, data_val, label_train, label_val = train_test_split(data2D, labels, test_size=0.1, random_state=42)
transformations = transforms.ToTensor()

train_data = TorchDataSet(data_train, label_train, transformations)
validation_data = TorchDataSet(data_val, label_val, transformations)

In [44]:
# input img shape = (num_imgs, 1, 28, 28)
# after first Conv2d = (num_imgs, 16, 28, 28)
# after first MaxPool2d layer = (num_imgs, 16, 14, 14)
# after second Conv2d = (num_imgs, 32, 14, 14)
# after second MaxPool2d layer = (num_imgs, 32, 7, 7)
# after flatten = (num_imgs, 32*7*7)
# i.e. input of first dense layer should be 32*7*7

class basicModel(nn.Module):
    def __init__(self, num_classes):
        super(basicModel, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, 5, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 5, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            nn.Flatten()
        )

        self.lin_layers = nn.Sequential(
            nn.Linear(32*7*7, 128),
            nn.Linear(128, 64),
            nn.Linear(64, num_classes),
        )

        # self.softmax = nn.LogSoftmax(num_classes)
    @profile
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.lin_layers(x)
        # x = self.softmax(x)

        return x

num_classes = len(np.unique(labels))
model = basicModel(num_classes)
loss_funk = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01)


# To Do:
--------------
- ReWrite train function
- call DataLoader on PytorchDataSet to load data into model
- specify batchsize, number of epoches and learning rate for Adam opt
<br><br><br><br>

In [45]:
num_epoch = 
train_losses = []
val_losses = []

def train(epoch):
    model.train()
    tr_loss = 0
    data_train, label_train = Variable(train_data), Variable(train_label)
    data_val, label_val = Variable(val_data), Variable(val_label)
    
    optimizer.zero_grad()
    
    output_train = model(train_data)
    output_val = model(val_data)
    
    loss_train = loss_funk(output_train, train_label)
    loss_val = loss_funk(output_val, val_label)
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    print('\n=====================================')
    print(f'Epoch: {epoch+1}\t loss: {loss_val}')
    print('=====================================')
    

In [48]:
for i in range(num_epoch):
    train(i)

TypeError: Variable data has to be a tensor, but got Subset